In [4]:
!pip install torch_geometric


In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T

dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=T.NormalizeFeatures())
data = dataset[0]



In [6]:
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [7]:
print(data.train_mask)

tensor([ True,  True,  True,  ..., False, False, False])


In [8]:
print(data.x)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [9]:
print(data.y)

tensor([3, 1, 5,  ..., 3, 1, 5])


In [10]:
print(f"Dataset: {dataset.name}")
print('-----------------------------')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of features per node: {dataset.num_node_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Training nodes: {data.train_mask.sum()}')
print(f'Validation nodes: {data.val_mask.sum()}')
print(f'Test nodes: {data.test_mask.sum()}')


Dataset: CiteSeer
-----------------------------
Number of nodes: 3327
Number of edges: 9104
Number of features per node: 3703
Number of classes: 6
Training nodes: 120
Validation nodes: 500
Test nodes: 1000


In [11]:
# model from paper
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x



In [12]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()
    
@torch.no_grad()
def evaluate():
    model.eval()
    logits = model(data)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].argmax(dim=1)
        acc = (pred == data.y[mask]).sum() / mask.sum()
        accs.append(acc.item())
    return accs  # train_acc, val_acc, test_acc


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, 16, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [14]:
best_val_acc = 0
best_test_acc = 0

for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = evaluate()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc
    if epoch % 20 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Train: {train_acc:.3f} | "
              f"Val: {val_acc:.3f} | Test: {test_acc:.3f}")

print('-----------------------------')
print(f'Best Validation Accuracy: {best_val_acc:.3f}')
print(f'Test Accuracy (at best val): {best_test_acc:.3f}')

Epoch 001 | Loss: 1.7919 | Train: 0.225 | Val: 0.076 | Test: 0.087
Epoch 020 | Loss: 1.6328 | Train: 0.942 | Val: 0.676 | Test: 0.659
Epoch 040 | Loss: 1.3818 | Train: 0.950 | Val: 0.688 | Test: 0.670
Epoch 060 | Loss: 1.1078 | Train: 0.950 | Val: 0.706 | Test: 0.675
Epoch 080 | Loss: 0.8990 | Train: 0.958 | Val: 0.706 | Test: 0.696
Epoch 100 | Loss: 0.7080 | Train: 0.950 | Val: 0.702 | Test: 0.695
Epoch 120 | Loss: 0.5745 | Train: 0.967 | Val: 0.706 | Test: 0.701
Epoch 140 | Loss: 0.5577 | Train: 0.975 | Val: 0.704 | Test: 0.696
Epoch 160 | Loss: 0.4931 | Train: 0.967 | Val: 0.704 | Test: 0.699
Epoch 180 | Loss: 0.4666 | Train: 0.975 | Val: 0.704 | Test: 0.706
Epoch 200 | Loss: 0.4711 | Train: 0.975 | Val: 0.694 | Test: 0.702
-----------------------------
Best Validation Accuracy: 0.718
Test Accuracy (at best val): 0.691


In [15]:
# train without edges
# basline results


from copy import deepcopy
data_without_edge = deepcopy(data)
data_without_edge.edge_index = torch.empty((2, 0), dtype=torch.long)
data = data_without_edge


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, 16, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


best_val_acc = 0
best_test_acc = 0

for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = evaluate()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc
    if epoch % 20 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Train: {train_acc:.3f} | "
              f"Val: {val_acc:.3f} | Test: {test_acc:.3f}")

print('-----------------------------')
print(f'Best Validation Accuracy: {best_val_acc:.3f}')
print(f'Test Accuracy (at best val): {best_test_acc:.3f}')

Epoch 001 | Loss: 1.7914 | Train: 0.433 | Val: 0.128 | Test: 0.167
Epoch 020 | Loss: 1.4684 | Train: 1.000 | Val: 0.452 | Test: 0.462
Epoch 040 | Loss: 1.0109 | Train: 1.000 | Val: 0.520 | Test: 0.509
Epoch 060 | Loss: 0.6963 | Train: 1.000 | Val: 0.522 | Test: 0.529
Epoch 080 | Loss: 0.5026 | Train: 1.000 | Val: 0.540 | Test: 0.543
Epoch 100 | Loss: 0.4987 | Train: 1.000 | Val: 0.532 | Test: 0.543
Epoch 120 | Loss: 0.4347 | Train: 1.000 | Val: 0.542 | Test: 0.538
Epoch 140 | Loss: 0.4686 | Train: 1.000 | Val: 0.558 | Test: 0.555
Epoch 160 | Loss: 0.3888 | Train: 1.000 | Val: 0.538 | Test: 0.551
Epoch 180 | Loss: 0.3856 | Train: 1.000 | Val: 0.562 | Test: 0.555
Epoch 200 | Loss: 0.3613 | Train: 1.000 | Val: 0.570 | Test: 0.571
-----------------------------
Best Validation Accuracy: 0.572
Test Accuracy (at best val): 0.571
